In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [2]:
from transformers import pipeline
qa=pipeline('question-answering')
context="""The platform has been quickly gaining market share in the US where, according to Fox, it has almost 80 million monthly active users.
In the UK, Tubi says it will offer more than 20,000 films and TV series, including content from Disney, Lionsgate, NBCUniversal and Sony Pictures Entertainment.
The platform will also include a selection of British, Indian and Nigerian content."""
question="How many active users each month?"
question_answer=qa(question=question,context=context)
start,end=question_answer['start'],question_answer['end']
print(context[start:end])

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recom

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

80 million


In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
checkpoint="distilbert-base-cased-distilled-squad"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)
model=AutoModelForQuestionAnswering.from_pretrained(checkpoint)
inputs=tokenizer(question,context,return_tensors="pt")
outputs=model(**inputs)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [4]:
start_logits=outputs.start_logits
end_logits=outputs.end_logits
print(start_logits.shape,end_logits.shape)

torch.Size([1, 89]) torch.Size([1, 89])


In [5]:
import torch

sequence_ids=inputs.sequence_ids()
mask=[i!=1 for i in sequence_ids]
mask[0]=False
mask=torch.tensor(mask)[None]

start_logits[mask]=-10000
end_logits[mask]=-10000

In [6]:
start_prob=torch.nn.functional.softmax(start_logits,dim=-1)[0]
end_prob=torch.nn.functional.softmax(end_logits,dim=-1)[0]

In [7]:
scores=start_prob[:,None]*end_prob[None,:]
scores=torch.triu(scores)

In [8]:
max_score=scores.argmax().item()
start_index=max_score//scores.shape[1]
end_index=max_score%scores.shape[1]
print(scores[start_index,end_index])

tensor(0.7146, grad_fn=<SelectBackward0>)


In [9]:
start_index,end_index

(29, 30)

In [16]:
inputs_with_offsets=tokenizer(question,context,return_offsets_mapping=True)
offsets=inputs_with_offsets["offset_mapping"]
start_char,_=offsets[start_index]
_,end_char=offsets[end_index]
answer=context[start_char:end_char]

result={
    'answer':answer,
    'start_index':start_index,
    'end_index':end_index,
    'probability':scores[start_index,end_index]
}
result

{'answer': '80 million',
 'start_index': 29,
 'end_index': 30,
 'probability': tensor(0.7146, grad_fn=<SelectBackward0>)}

In [18]:
inputs=tokenizer(question,context)
print(len(inputs['input_ids']))

89


In [31]:
context='''Last month, AI chip giant Nvidia briefly became the world’s richest company, overtaking Microsoft, which had in turn risen above Apple.

When this news was mentioned on stage at a tech industry event I attended in Copenhagen, there was spontaneous applause from the audience.

As I write, Nvidia is now back in second place, after a fall in its share price took its combined value down to $3tn (£2.4tn) compared with $3.4tn for Microsoft.

Two things have propelled these two US tech titans to such a dizzying pinnacle: AI and foresight.

Microsoft started investing in OpenAI, the creator of popular AI chatbot ChatGPT, back in 2019. Meanwhile, Nvidia boss Jensen Huang pushed his company towards AI chip development many years before generative AI exploded onto the scene.

Both firms took a long-term bet on the current AI boom – and so far, it’s paid off, leaving former top-dog Apple trailing in their wake. But how long will it last?
Last month, AI chip giant Nvidia briefly became the world’s richest company, overtaking Microsoft, which had in turn risen above Apple.

When this news was mentioned on stage at a tech industry event I attended in Copenhagen, there was spontaneous applause from the audience.

As I write, Nvidia is now back in second place, after a fall in its share price took its combined value down to $3tn (£2.4tn) compared with $3.4tn for Microsoft.

Two things have propelled these two US tech titans to such a dizzying pinnacle: AI and foresight.

Microsoft started investing in OpenAI, the creator of popular AI chatbot ChatGPT, back in 2019. Meanwhile, Nvidia boss Jensen Huang pushed his company towards AI chip development many years before generative AI exploded onto the scene.

Both firms took a long-term bet on the current AI boom – and so far, it’s paid off, leaving former top-dog Apple trailing in their wake. But how long will it last?

This year’s London Tech Week, an annual event for the UK tech scene, may as well have been called London AI Week. The letters AI were emblazoned on every stand, and uttered in every speech.

I bumped into Anne Boden, the founder of Starling Bank, a significant fintech disruptor. She was buzzing with excitement.

“We thought we knew who the winners and losers were [in tech],” she told me. “But with AI, we are throwing the dice again”.

She believes she’s watching the AI revolution re-landscape the tech sector, and she wants to dive back in.

That same week I also popped along to Founders Forum, an annual gathering of around 250 high-level entrepreneurs and investors. Some serious money, in other words. It’s a confidential event, but I don’t think I’ll get into too much trouble for saying that much of the chat there was also centred around AI.'''
question="Who is Anne Boden and what is her association with the tech industry?"
inputs=tokenizer(question,context)
print(len(inputs['input_ids']))

657


In [32]:
inputs=tokenizer(question,context,max_length=384,truncation='only_second')
tokenizer.decode(inputs['input_ids'])

'[CLS] Who is Anne Boden and what is her association with the tech industry? [SEP] Last month, AI chip giant Nvidia briefly became the world ’ s richest company, overtaking Microsoft, which had in turn risen above Apple. When this news was mentioned on stage at a tech industry event I attended in Copenhagen, there was spontaneous applause from the audience. As I write, Nvidia is now back in second place, after a fall in its share price took its combined value down to $ 3tn ( £2. 4tn ) compared with $ 3. 4tn for Microsoft. Two things have propelled these two US tech titans to such a dizzying pinnacle : AI and foresight. Microsoft started investing in OpenAI, the creator of popular AI chatbot ChatGPT, back in 2019. Meanwhile, Nvidia boss Jensen Huang pushed his company towards AI chip development many years before generative AI exploded onto the scene. Both firms took a long - term bet on the current AI boom – and so far, it ’ s paid off, leaving former top - dog Apple trailing in their 

In [33]:
inputs=tokenizer(
    question,
    context,
    max_length=384,
    stride=128,
    padding='longest',
    truncation='only_second',
    return_overflowing_tokens=True,
    return_offsets_mapping=True)


In [34]:
_=inputs.pop('overflow_to_sample_mapping')
offsets=inputs.pop('offset_mapping')
inputs=inputs.convert_to_tensors('pt')
print(inputs['input_ids'].shape)

torch.Size([3, 384])


In [35]:
outputs=model(**inputs)
start_logits=outputs.start_logits
end_logits=outputs.end_logits
print(start_logits.shape,end_logits.shape)

torch.Size([3, 384]) torch.Size([3, 384])


In [36]:
inputs['attention_mask']

tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])

In [37]:
sequence_ids=inputs.sequence_ids()
mask=[i!=1 for i in sequence_ids]
mask[0]=False
mask=torch.logical_or((torch.tensor(mask)[None]),inputs['attention_mask']==0)
start_logits[mask]=-10000
end_logits[mask]=-10000

In [38]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

In [39]:
start_probabilities

tensor([[9.6476e-01, 0.0000e+00, 0.0000e+00,  ..., 1.0232e-05, 9.9342e-06,
         0.0000e+00],
        [1.0584e-03, 0.0000e+00, 0.0000e+00,  ..., 3.0460e-08, 3.8980e-08,
         0.0000e+00],
        [6.0397e-06, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], grad_fn=<SoftmaxBackward0>)

In [41]:
candidates=[]
for start_prob, end_prob in zip(start_probabilities, end_probabilities):
    scores=start_prob[:,None]*end_prob[None,:]
    idx=torch.triu(scores).argmax().item()
    start_ind=idx//scores.shape[1]
    end_ind=idx%scores.shape[1]
    score=scores[start_ind,end_ind]
    candidates.append((start_ind,end_ind,score.item()))
print(candidates)

[(0, 0, 0.9286502003669739), (271, 276, 0.3479771018028259), (33, 38, 0.6577988862991333)]
